# Redistricting Lab

**Objectives**
 - Introduce students to congressional redistricting and gerrymandering
 - Introduce students to a new integer programming formulation
 - Expose students to measures of compactness and the efficiency gap
 - Give students experience creating congressional districts for differet objectives

**Reading:** This lab is based on real congressional redistricting research done by Wes Gurnee and David Shmoys. More information about this research can be found at [fairmandering.org](https://www.fairmandering.org). A more complete description of the algorithm can be found [here](https://www.fairmandering.org/algorithm.html).

**Brief description:** In this lab, we will consider a section of a new paper on congressional redistricting. After summarizing the problem of congressional redistricting, we will give a breif overview of the the paper leading up to the relevant section. Lastly, we will explore possible districts for the state of Georgia subject to various objectives.

 <br>


In [ ]:
# imports -- don't forget to run this cell!
from redistricting import *
from ortools.linear_solver import pywraplp as OR
%matplotlib inline

# Part I: Redistricting, Gerrymandering, and the Efficiency Gap

Every 10 years in the US, following the census, the process of *reapportionment* determines how the 435 representatives in the House of Representatives will be distributed among the 50 states. Depending on population shifts, a state may lose or gain a representative. Each state is divided in to electoral districts; one for each representative. Hence, after reapportionment, states *redistrict* to redraw the electoral district boundaries. These boundaries are drawn subject to a few federal constraints:

- Districts must have roughly the same population (total state population / number of representatives).
- They must be compact and contiguous.
- You can not divide minorities across multiple districts (cracking) or have a district in which they are the majority (packing).

You may have noticed that there is no restriction on packing or cracking political parties. These two strategies are common in a phenomenon known as *gerrymandering*. Gerrymandering is the process of redrawing these electoral district boundaries to give one political party an advantage over another. In the diagram below, you can see an example of both cracking in packing. 

<img src="images/gerrymandering.png" width=600 height=300 />

*Taken from [The Washington Post](https://www.washingtonpost.com/news/wonk/wp/2015/03/01/this-is-the-best-explanation-of-gerrymandering-you-will-ever-see/)*

Each district shoud have 10 squares. On the right, the two U-shaped districts are packed with blue squares (9/10). The remaining blue squares are then cracked by splitting them among the remaining districts. Even though red is the minority, gerrymandering allow them to have a majority of the districts.

To address this, some states have introduced restrictings aganist this. However, it is hard to police. How can you prove that someone gerrymandered or not? We will look at one fairness metric that has received a lot of attention called the *efficiency gap*. Essentially, the efficiency gap tracks to types of "wasted" votes: lost votes for the losing canidate and excess votes (above 50%) for the winning canidate. 

**Q1:** Consider each square in diagram (3) above a vote. How many wasted votes does blue have? How many are lost votes and how many are excess votes? How does this compare to red?

**A:** 

Now that we know the total wasted votes for each party, we can compute the efficiency gap as follows:

$$\text{Efficiency Gap} = \frac{\text{Total Wasted Blue Votes} - \text{Total Wasted Red Votes}}{\text{Total Votes}}$$

**Q2:** Continuing with our example, what is the efficiency gap?

**A:** 

You should have found the efficiency gap to be $0.30$. This has the following interpretation: the red party won $30\%$ more seats than they should have. $30\%$ of $5$ is $1.5$. They should have won 2 seats but they managed to win an additional seat. If everything were perfectly fair, the efficiency gap would be 0. However, perfect equality is impossible. The authors of the efficiency gap, Stephanopoulos and McGhee, propose $\pm2\%$ for congressional redistricting.

## Part II: Fairmandering: How to Redistrict Fairly



Each state is divided in to census tracts with a population of approximatly 4,000 for the purpose of taking the census. Given a number of representatives $k$, redistricting consists of assigning each of these census tracts to a district subject to compactness and continuity. Subject to these constraints, we could optimize by minimizing the absolute value of the efficiency gap. However, the scale prevents us from formulating a single IP to tackle this whole problem. Below, we load the census tracts for Georgia.

In [ ]:
tracts = load_tract_shapes()
tracts.plot(figsize=(10, 10), color='none', edgecolor='black', lw=.25);
print('There are %d census tracts in Georgia' % (len(tracts)))

To keep track of continuity, we can create an adjacency graph where there is a node for each census tract and an edge between two nodes if and only if those census tracts are geographically adjacent.

In [ ]:
adjacency_graph = load_graph()
draw_adjacency_graph(tracts, adjacency_graph, figsize=(10, 10));
print('The adjacency graph has %d nodes and %d edges.' % (len(adjacency_graph.nodes), len(adjacency_graph.edges)))

Using this adjacency graph, we can generate a list of all feasible districts. We now have a set of census tracts $i \in T$ and a set of feasible districts $j \in D$. Furthermore, we have a matrix $A$ where $a_{ij} = 1$ if and only if census tract $i$ is contained within district $j$. We can compute the efficiency gap $c_j$ for each feasible district $j \in D$. The following formulation then chooses a final set of $k$ districts:

$$\begin{align*}
\min \quad & \left| \enspace \sum_{j \in D} c_jx_j \enspace \right| \\
\text{s.t.} \quad &  \sum_{j \in D} a_{ij}x_j = 1 \quad \forall i \in T & (1)\\
\quad &  \sum_{j \in D}  x_j = k & (2)\\
\quad & x_j \in \{0,1\} \quad \forall j \in D & (3)\\
\end{align*}$$

**Q3:** What are the decision variables? What does their value represent?

**A:** 

**Q4:** What does constraint (1) enforce?

**A:** 

**Q5:** What does constraint (2) enforce?

**A:** 

**Q6:** Is this formulation an integer linear program? Why or why not?

**A:** 

It turns out there is a special trick for encoding an absolute value in a linear program. Consider the simpler example of maximizing $x$. We can introduce a new variable $w$ and enforce the constraints $x \leq w$ and $x \geq -w$. Now, we can just minimize $w$ as our objective function. Do you see why this works? After introducing this trick into our formulation, we get the following integer linear program (ILP):

$$\begin{align*}
\min \quad & w \\
\text{s.t.} \quad &  \sum_{j \in D} a_{ij}x_j = 1 \quad \forall i \in T & (1)\\
\quad &  \sum_{j \in D}  x_j = k & (2)\\
\quad & \sum_{j \in D} c_jx_j \leq w & (3) \\
\quad & \sum_{j \in D} c_jx_j \geq -w & (4) \\
\quad & x_j \in \{0,1\} \quad \forall j \in D & (5)\\
\end{align*}$$

**Q7:** How many variables does this ILP have? Do you think this ILP is reasonable as is for a state with nearly 2000 census tracts? (Hint: How many feaisble districts do you think that would generate?)

**A:** 

The paper by Wes Gurnee and David Shmoys addressee this very issue! They formualated a structured way of generating a large number of compatible feasible districts (though small relative to the total number possible). We won't go in to detail about this process but here is a breif overview.

Esssentially, the state is divided into some number of smaller regions (partitioned) some number of times (385 in the case of this Georgia example). For each of these partitions, this process is repeated until the size of the regions is appropriate for the size of a district. Here is a small section of sample tree: 

<img src="images/sample_tree.png" width=650 height=300 />

*Taken from [fairmandering.org](https://www.fairmandering.org/algorithm.html)*

The root of this tree is the entire state. In the first layer of the tree, you have the first partitions (which we call root partitions). In the last layer of the tree (also called the leaf nodes), the partions represent the feasible districts. Just one more piece of vocabulary! If you can follow a path directly down from one node $a$ to another $b$, we say that $a$ is a ancestor of $b$. For example, the middle root partition is an ancestor of every shown leaf. The right and left root partitions are **not** ancestors of any shown leaf.

**Q8:** Can two districts who have different root partition ancestors be used in the same plan together? Why or why not?

**A:** 

In **Q8**, you should have argued that they would not be compatible. Hence, we solve this optimization problem for each of the root partitions. Each root partition is the ancestor of some subset of the leafs which represent feasible districts. For each root partition, we can use these feasible districts as the input to the ILP we described! Let's try it out!

## Part III: Redistricting Georgia

First, let's look at some of the data we have. The dataframes below give us some statistics and past election results for each census tract.

In [ ]:
tract_stats = load_state_df()
display(tract_stats.head())
tract_results = load_election_df()
display(tract_results.head())

In [ ]:
# Plot 2016 presidential election results
tracts['2016_pres'] = tract_results.apply(lambda x: x.R_pres_2016 / (x.R_pres_2016 + x.D_pres_2016), axis=1)
tracts.plot(column='2016_pres', cmap='seismic', vmin=0, vmax=1, figsize=(15, 10));

We skipped a lot of complexity involved in generating a sample tree. For the sake of this lab, this data has been given to us.

In [ ]:
leaf_nodes = pickle.load(open('data/leaf_nodes.p', 'rb'))
internal_nodes = pickle.load(open('data/internal_nodes.p', 'rb'))
print('There are %d root partitions and %d feasible districts.' % (len(internal_nodes[0]['children_ids']), 
                                                                   len(leaf_nodes)))

In [ ]:
# change i (0..215784) to highlight different feasible districts corresponding to the leaf nodes
i = 0
ax = tracts.plot(figsize=(10, 10), color='none', edgecolor='black', lw=.25) 
tracts.iloc[leaf_nodes[i]['area']].plot(ax=ax, color='green', alpha=.3);

For each feasible district, we can aggregate over the census tract statistics contained within it to get statistics for each feasible district. In addition to the aggregate statistics, we have also computed some measures of compactness: `roeck`, `edge_ratio`, and `cut_edges` (which we will discuss later) and the efficiency gap for each district.

In [ ]:
district_df = pd.read_csv(os.path.join('data', 'ga_district_df.csv'))
district_df['efficiency_gap'] = efficiency_gap_coefficients(district_df, .53)
display(district_df.head())

**Q9:** Complete the model below by implementing the $k$ total districts constraint.

In [ ]:
def redistrict(k, tract_district_matrix, cost, integer=False, opt_type='abs_val', solver='CBC'):
    """A model for solving a congressional redistricting problem.
    
    Args:
        k (int): number of districts in a plan
        tract_district_matrix (np.ndarray): binary matrix a_ij = 1 if tract i is in district j
        costs (np.ndarray): cost coefficients of districts
        opt_type (str): {"minimize", "maximize", "abs_val"}
        solver (str) : {"CBC", "gurobi"}
    """
    n_tracts, n_columns = tract_district_matrix.shape
    TRACTS = range(n_tracts)
    DISTRICTS = range(n_columns)
    
    # define the model
    if solver=='CBC':
        m = OR.Solver('redistrict', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)        
    elif solver=='gurobi':
        m = OR.Solver('redistrict', OR.Solver.GUROBI_MIXED_INTEGER_PROGRAMMING)
    else:
        raise ValueError('Invalid solver')

    # decision variables
    x = {} # x_i is 1 if district i is used, 0 otherwise
    for d in DISTRICTS:
        if integer:
            x[d] = m.IntVar(0, 1, name="x(%s)" % d)
        else:
            x[d] = m.NumVar(0, 1, name="x(%s)" % d)

    # objective function
    if opt_type == 'min':
        m.Minimize(sum(cost[d] * x[d] for d in DISTRICTS))
    elif opt_type == 'max':
        m.Maximize(sum(cost[d] * x[d] for d in DISTRICTS))
    elif opt_type == 'abs_val':
        w = m.NumVar(-k, k, name="w")
        m.Add(sum(cost[d] * x[d] for d in DISTRICTS) <= w, name='absval_pos')
        m.Add(sum(cost[d] * x[d] for d in DISTRICTS) >= -w, name='absval_neg')
        m.Minimize(w)
    else:
        raise ValueError('Invalid optimization type')

    # subject to: each census tract appears in exactly one district
    for t in TRACTS:    
        m.Add(sum(x[d] * tract_district_matrix[t,d] for d in DISTRICTS) == 1)

    # subject to: k total districts
    # TODO: Implement this constraint
    
    

    return m,x

In [ ]:
# You do not need to pay attention to this cell but make sure you run it!
def solve(m, solver='CBC'):
    """Solve the model and specify some solver parameters."""
    if solver=='CBC':
        m.SetTimeLimit(int(len(leaf_nodes) / 10))
        params = OR.MPSolverParameters()
        params.SetDoubleParam(params.RELATIVE_MIP_GAP, 1e-4)
        status = m.Solve(params)
    elif solver=='gurobi':
        params_set = m.SetSolverSpecificParametersAsString(
                     '''TimeLimit %d
                        MIPGapAbs %d''' % ((len(leaf_nodes) / 10), 1e-4))
        if params_set:
            print('Gurobi solver parameters set successfully.')
        status = m.Solve()
    if status == OR.Solver.OPTIMAL:
        print('Optimal solution found.')
        print('Objective value =', m.Objective().Value())
    else:
        print('Optimal solution was not found - %s' % (status))

Before running our model, we first need to format our data to fit the models input. We want to split Georgia into 14 districts. The matrix $A$ is equivalent to `tract_district_matrix`. Lastly, we get a map from root partitions to the relevant leaf nodes; each representing a problem instance.

In [ ]:
n_districts = 14
tract_district_matrix = make_tdm(leaf_nodes, len(tracts))
partition_map = list(make_root_partition_to_leaf_map(leaf_nodes, internal_nodes).items())

Unfortunately, this problem is still large enough that the open-source solver we use in this class (CBC) would take a little over an hour to solve all 385 root partitions. For that reason, we will solve a small subset of these root partitions. The function below solves some subset of the root partitions for given cost coefficents and an optimization type.

In [ ]:
def solve_root_partitions(cost, opt_type, index_subset):
    '''Solve a subset of the root partitions and return a list of each solution.'''
    solutions = []
    for i in index_subset:
        partition_ix, leaf_slice = partition_map[i]
        m,x = redistrict(k=n_districts,
                         tract_district_matrix=tract_district_matrix[:, leaf_slice],
                         cost=np.array(cost)[leaf_slice],
                         integer=True,
                         opt_type=opt_type,
                         solver='CBC')
        solve(m, solver='CBC')
        opt_cols = [j for j, v in x.items() if v.solution_value() > .5]
        solutions.append({'n_leaves': len(leaf_slice),
                          'solution_ixs': leaf_slice[opt_cols],
                          'optimal_objective': np.array(cost)[leaf_slice][opt_cols]})
    return solutions

We want to minimize the absolute value of the efficency gap!

In [ ]:
solutions = solve_root_partitions(district_df['efficiency_gap'], 'abs_val', [4,5,6])

Now that we have some solutions, we need to pick one of them. The simplest thing to do is pick the one with the smallest efficiency gap.

In [ ]:
for sol in solutions:
    e_gap = round(district_df.loc[sol['solution_ixs']]['efficiency_gap'].mean(),5)
    print(e_gap)

**Q10:** Just by looking at the efficiency gap for each of these solutions, which of these is the most fair? 

**A:** 

**Q11:** In the third solution, the efficiency gap is nearly $0.04$. For which party is this an advantage?

**A:** 

Let's look at each solution graphically since there are only 3!

In [ ]:
for sol in solutions:
    politics_map(tracts, district_df, leaf_nodes, sol, figsize=(6,6));

**Q12:** Do these solutions reflect what you would expect from the efficiency gaps?

**A:** 

## Part IV: Redistricting Georgia (Different Objectives)

In **Part III** we minimized the absolute value of the efficiency gap to make things fair. Alternatively, we could use this same tool to gerrymander. For example, if we wanted to gerrymander to favor The Republican Party, we would want to maximize the efficiency gap. Let's do this for the same root partitions we optimized to be fair.

In [ ]:
solutions = solve_root_partitions(district_df['efficiency_gap'], 'max', [4,5,6])

In [ ]:
for sol in solutions:
    politics_map(tracts, district_df, leaf_nodes, sol, figsize=(6,6));
    e_gap = round(district_df.loc[sol['solution_ixs']]['efficiency_gap'].mean(),5)
    print(e_gap)

**Q13:** How effective was the gerrymandering? How did the most fair solution (6 D, 8 R seats) compare to solution with the highest efficiency gap here? Based on the interpretation of the efficiency gap, does this make sense?

**A:** 

**Q14:** How was the Democratic vote supressed so heavily?

**A:** 

**Q15:** Choose the correct cost coefficents and optimization type to gerrymander for The Democratic Party 

**Note**: We look at a different 3 root partitions here because the previous 3 are hard to gerrymander in the The Democratic Party's favor.

In [ ]:
# TODO: Uncomment, and gerrymander for The Democratic Party
# solutions = solve_root_partitions(, , [4,110,216])



In [ ]:
for sol in solutions:
    politics_map(tracts, district_df, leaf_nodes, sol, figsize=(6,6));
    e_gap = round(district_df.loc[sol['solution_ixs']]['efficiency_gap'].mean(),5)
    print(e_gap)

**Q16:** How effective was the gerrymandering? How did the most fair solution (6 D, 8 R seats) compare to solution with the highest efficiency gap here? Based on the interpretation of the efficiency gap, does this make sense?

**A:** 

**Q17:** How was the Republican vote supressed so heavily?

**A:** 

Up to this point, we have focused on optimizing around partisanship. However, we could also optimize around other measures like compactness. In  order to do this, we need to decide how to measure compactness. One possible measure is called the Roeck test. First, the smallest circle enclosing the district is found. Then, the area of the district is divded by the area of that circle. More compact districts will have a measure closer to 1. Let's look at some examples.

In [ ]:
for i in [40339, 215783, 107817]:
    ax = tracts.plot(figsize=(6, 6), color='none', edgecolor='black', lw=.25) 
    tracts.iloc[leaf_nodes[i]['area']].plot(ax=ax, color='green', alpha=.3);
    print('Roeck measure: %f' % (district_df.loc[i]['roeck']))

**Q18:** Choose the correct cost coefficents and optimization type to maximize the compactness of the districts using the Roeck test. (Hint: the column `roeck` gives the Roeck measure for each district)

In [ ]:
# TODO: Uncomment, and maximize the compactness of the districts
# solutions = solve_root_partitions(, , [4,5,6])



In [ ]:
for sol in solutions:
    politics_map(tracts, district_df, leaf_nodes, sol, figsize=(6,6));
    roeck = round(district_df.loc[sol['solution_ixs']]['roeck'].mean(),5)
    print(roeck)

**Q19:** Which party does maximizing compactness benefit? Why?

**A:** 

Another measure of compactness one could use is the number of cut edges. Think back to the adjacency graph. The number of cut edges is the number of edges one would have to remove to make the district a separate component from the rest of the graph. The more cut edges, the less compact the distict is.

**Q20:** Choose the correct cost coefficents and optimization type to minimize the compactness of the districts using cut edges. (Hint: the column `cut_edges` gives the number of cut edges for each district)

In [ ]:
# TODO: Uncomment, and maximize the compactness of the districts
# solutions = solve_root_partitions(, , [4,5,6])



In [ ]:
for sol in solutions:
    politics_map(tracts, district_df, leaf_nodes, sol, figsize=(6,6));
    cut_edges = round(district_df.loc[sol['solution_ixs']]['cut_edges'].mean(),5)
    print(cut_edges)